<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://cloud.mail.ru/public/3bwi/bFYHDN5S5) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [ ]:
!head -1 ../../data/stackoverflow.10kk.tsv

Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [ ]:
%%time
!wc -l ../../data/stackoverflow.10kk.tsv

Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [ ]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [1]:
%%time
!python preprocess.py ../../data/stackoverflow.10kk.tsv ../../data/stackoverflow.vw

CPU times: user 2.34 s, sys: 256 ms, total: 2.6 s
Wall time: 2min 13s


In [2]:
!wc -l ../../data/stackoverflow.vw

4389054 ../../data/stackoverflow.vw


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

''' ВАШ КОД ЗДЕСЬ '''
!head -1463018 ../../data/stackoverflow.vw > ../../data/stackoverflow_train.vw
!sed -i '1,+1463017d' ../../data/stackoverflow.vw

!wc -l ../../data/stackoverflow_train.vw
!wc -l ../../data/stackoverflow.vw

%%time
!head -1463018 ../../data/stackoverflow.vw > ../../data/stackoverflow_valid.vw
!sed -i '1,+1463017d' ../../data/stackoverflow.vw

!wc -l ../../data/stackoverflow_valid.vw
!wc -l ../../data/stackoverflow.vw

mv ../../data/stackoverflow.vw ../../data/stackoverflow_test.vw

In [3]:
!split -l 1463018 ../../data/stackoverflow.vw ../../data/stackoveflow_segment

In [7]:
!ls -al ../../data

total 21009852
drwxrwxr-x 5 borowis borowis       12288 May  2 00:44 .
drwxrwxr-x 7 borowis borowis        4096 Apr 21 23:47 ..
-rw-rw-r-- 1 borowis borowis     3518607 Feb 28 20:27 adult.data.csv
-rw-rw-r-- 1 borowis borowis     2096597 Mar 18 04:45 adult_test.csv
-rw-rw-r-- 1 borowis borowis     3835686 Mar 18 04:45 adult_train.csv
-rwxrwxr-x 1 borowis borowis     3361468 Apr 21 23:47 bank_train.csv
-rwxrwxr-x 1 borowis borowis       55190 Apr 21 23:47 bank_train_target.csv
-rw-rw-r-- 1 borowis borowis       31107 Feb 28 20:27 beauty.csv
-rw-rw-r-- 1 borowis borowis     1812928 Apr  2 22:40 credit_scoring_sample.csv
-rw-rw-r-- 1 borowis borowis     3859157 Feb 28 20:27 credit_scoring_test.csv
-rw-rw-r-- 1 borowis borowis     3948401 Feb 28 20:27 credit_scoring_train.csv
-rw-rw-r-- 1 borowis borowis         100 Feb 28 20:27 drugs-and-math.csv
drwxrwxr-x 3 borowis borowis        4096 Apr 16 00:28 faces
-rw-rw-r-- 1 borowis borowis       17063 Feb 28 20:27 girls.csv
-rwxrwxr-x 1 borowis

In [11]:
!wc -l ../../data/stackoveflow_segmentaa
!wc -l ../../data/stackoveflow_segmentab
!wc -l ../../data/stackoveflow_segmentac

1463018 ../../data/stackoveflow_segmentaa
1463018 ../../data/stackoveflow_segmentab
1463018 ../../data/stackoveflow_segmentac


In [13]:
f_out_labels = open("../../data/stackoveflow_segmentab_labels.txt", 'w')
with open("../../data/stackoveflow_segmentab") as f:
    for line in f:
        splitted = line.split("|")
        
        f_out_labels.write(str(splitted[0].strip()) + "\n")
        
f_out_labels.close()

In [14]:
!wc -l ../../data/stackoveflow_segmentab_labels.txt

1463018 ../../data/stackoveflow_segmentab_labels.txt


In [15]:
f_out_labels = open("../../data/stackoveflow_segmentac_labels.txt", 'w')
with open("../../data/stackoveflow_segmentac") as f:
    for line in f:
        splitted = line.split("|")
        
        f_out_labels.write(str(splitted[0].strip()) + "\n")
        
f_out_labels.close()

In [16]:
!wc -l ../../data/stackoveflow_segmentac_labels.txt

1463018 ../../data/stackoveflow_segmentac_labels.txt


### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [20]:
%%time

''' ВАШ КОД ЗДЕСЬ '''
!vw --oaa 10 --passes 1 --ngram 1 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_1_1.vw
!vw --oaa 10 --passes 1 --ngram 2 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_1_2.vw
!vw --oaa 10 --passes 1 --ngram 3 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_1_3.vw

!vw --oaa 10 -c --cache_file ../../data/stackoverflow_3_1_cache --passes 3 --ngram 1 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_3_1.vw
!vw --oaa 10 -c --cache_file ../../data/stackoverflow_3_2_cache --passes 3 --ngram 2 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_3_2.vw
!vw --oaa 10 -c --cache_file ../../data/stackoverflow_3_3_cache --passes 3 --ngram 3 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_3_3.vw

!vw --oaa 10 -c --cache_file ../../data/stackoverflow_5_1_cache --passes 5 --ngram 1 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_5_1.vw
!vw --oaa 10 -c --cache_file ../../data/stackoverflow_5_2_cache --passes 5 --ngram 2 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_5_2.vw
!vw --oaa 10 -c --cache_file ../../data/stackoverflow_5_3_cache --passes 5 --ngram 3 -d ../../data/stackoveflow_segmentaa -b 28 --random_seed 17 -f ../../data/stackoverflow_model_5_3.vw

Generating 1-grams for all namespaces.
final_regressor = ../../data/stackoverflow_model_1_1.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/stackoveflow_segmentaa
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.812500 0.875000           16           16.0        7        7      209
0.812500 0.812500           32           32.0        7        5      174
0.781250 0.750000           64           64.0        3        3      204
0.671875 0.562500          128          128.0        1        5       29
0.609375 0.546875     

In [21]:
%%time
!vw -t -i ../../data/stackoverflow_model_1_1.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_1_1.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_1_2.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_1_2.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_1_3.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_1_3.predictions --random_seed 17 

!vw -t -i ../../data/stackoverflow_model_3_1.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_3_1.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_3_2.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_3_2.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_3_3.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_3_3.predictions --random_seed 17 

!vw -t -i ../../data/stackoverflow_model_5_1.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_5_1.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_5_2.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_5_2.predictions --random_seed 17 
!vw -t -i ../../data/stackoverflow_model_5_3.vw -d ../../data/stackoveflow_segmentab -p ../../data/stackoverflow_model_5_3.predictions --random_seed 17 

Generating 1-grams for all namespaces.
only testing
predictions = ../../data/stackoverflow_model_1_1.predictions
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/stackoveflow_segmentab
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2      178
0.000000 0.000000            2            2.0        7        7       74
0.000000 0.000000            4            4.0        5        5      259
0.000000 0.000000            8            8.0        7        7      144
0.000000 0.000000           16           16.0        6        6      359
0.000000 0.000000           32           32.0        2        2      400
0.062500 0.125000           64           64.0        5        5     1061
0.070312 0.078125          128          128.0        2        2      132
0.08

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score
test_labels = np.loadtxt('../../data/stackoveflow_segmentab_labels.txt')
model_predictions = ['stackoverflow_model_1_1.predictions', 'stackoverflow_model_1_2.predictions', 'stackoverflow_model_1_3.predictions', \
                     'stackoverflow_model_3_1.predictions', 'stackoverflow_model_3_2.predictions', 'stackoverflow_model_3_3.predictions', \
                     'stackoverflow_model_5_1.predictions', 'stackoverflow_model_5_2.predictions', 'stackoverflow_model_5_3.predictions']
for model in model_predictions:
    vw_pred = np.loadtxt('../../data/' + model)
    print (model + ': ' + "{:2.16f}".format(accuracy_score(test_labels, vw_pred)))

stackoverflow_model_1_1.predictions: 0.9151541539475249
stackoverflow_model_1_2.predictions: 0.9310835546794366
stackoverflow_model_1_3.predictions: 0.9286030657175783
stackoverflow_model_3_1.predictions: 0.9143694746066009
stackoverflow_model_3_2.predictions: 0.9278012984119129
stackoverflow_model_3_3.predictions: 0.9263877819685062
stackoverflow_model_5_1.predictions: 0.9136538306432320
stackoverflow_model_5_2.predictions: 0.9292353204130093
stackoverflow_model_5_3.predictions: 0.9261670054640476


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- ** Биграммы и 1 проход по выборке **
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [23]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
!vw -t -i ../../data/stackoverflow_model_1_2.vw -d ../../data/stackoveflow_segmentac -p ../../data/stackoverflow_model_1_2_test.predictions --random_seed 17 

Generating 2-grams for all namespaces.
only testing
predictions = ../../data/stackoverflow_model_1_2_test.predictions
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/stackoveflow_segmentac
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.000000 0.000000            4            4.0        5        5      158
0.000000 0.000000            8            8.0        2        2      116
0.000000 0.000000           16           16.0        6        6       74
0.031250 0.062500           32           32.0        1        1      324
0.046875 0.062500           64           64.0        4        4       52
0.054688 0.062500          128          128.0        7        7      250

In [24]:
test_labels = np.loadtxt('../../data/stackoveflow_segmentac_labels.txt')
vw_pred = np.loadtxt('../../data/stackoverflow_model_1_2_test.predictions')
print (accuracy_score(test_labels, vw_pred))

0.931190867098


0.9310835546794366

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- ** Результаты почти одинаковы – отличаются меньше чем на 0.5% **

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [28]:
''' ВАШ КОД ЗДЕСЬ '''
!cat ../../data/stackoveflow_segmentaa ../../data/stackoveflow_segmentab > ../../data/stackoveflow_segmentaab
!wc -l ../../data/stackoveflow_segmentaab
!vw --oaa 10 --passes 1 --ngram 2 -d ../../data/stackoveflow_segmentaab -b 28 --random_seed 17 -f ../../data/stackoverflow_model_full.vw
!vw -t -i ../../data/stackoverflow_model_full.vw -d ../../data/stackoveflow_segmentac -p ../../data/stackoverflow_model_full_test.predictions --random_seed 17 

2926036 ../../data/stackoveflow_segmentaab
Generating 2-grams for all namespaces.
final_regressor = ../../data/stackoverflow_model_full.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../../data/stackoveflow_segmentaab
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.765625 0.750000           64           64.0        3        3      406
0.664062 0.562500          128          128.0     

In [29]:
test_labels = np.loadtxt('../../data/stackoveflow_segmentac_labels.txt')
vw_pred = np.loadtxt('../../data/stackoverflow_model_full_test.predictions')
print (accuracy_score(test_labels, vw_pred))

0.935186716773


0.9310835546794366

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - ** 0.4% **
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)